## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

## Importing Data

In [3]:
df = pd.read_csv('loan_data.csv')

In [4]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Checking null values first

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
## Getting the number of NULL values in each column
df.isnull().sum().sort_values()

Loan_ID               0
CoapplicantIncome     0
ApplicantIncome       0
Education             0
Loan_Status           0
Property_Area         0
Married               3
Gender               13
Loan_Amount_Term     14
Dependents           15
LoanAmount           22
Self_Employed        32
Credit_History       50
dtype: int64

In [7]:
## 1 -- Married column - Null value fill

In [8]:
## checking for mode because
# yes/no --> mode.  mean or median is considered only when the column has numerical values
df['Married'].value_counts(normalize=True)*100

Married
Yes    65.139116
No     34.860884
Name: proportion, dtype: float64

In [9]:
df['Married'].mode()[0]

'Yes'

In [10]:
df['Married'].value_counts(dropna=False)

Married
Yes    398
No     213
NaN      3
Name: count, dtype: int64

In [11]:
df['Married'].fillna(df['Married'].mode()[0],inplace=True)

In [12]:
df['Married'].value_counts(dropna=False)

Married
Yes    401
No     213
Name: count, dtype: int64

In [13]:
df.isnull().sum().sort_values()

Loan_ID               0
Married               0
ApplicantIncome       0
Education             0
CoapplicantIncome     0
Property_Area         0
Loan_Status           0
Gender               13
Loan_Amount_Term     14
Dependents           15
LoanAmount           22
Self_Employed        32
Credit_History       50
dtype: int64

In [14]:
## 2 -- Gender column - NULL value filling

In [15]:
df['Gender'].value_counts(dropna=False)

Gender
Male      489
Female    112
NaN        13
Name: count, dtype: int64

In [16]:
## only 2 categories Male/Female (string) so considering MODE to fill the NULL

In [17]:
df['Gender'].mode()[0]

'Male'

In [18]:
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)

In [19]:
df['Gender'].value_counts(dropna=False)

Gender
Male      502
Female    112
Name: count, dtype: int64

In [20]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Education             0
CoapplicantIncome     0
ApplicantIncome       0
Loan_Status           0
Property_Area         0
Loan_Amount_Term     14
Dependents           15
LoanAmount           22
Self_Employed        32
Credit_History       50
dtype: int64

In [21]:
## 3 -- Loan_Amount_Term is numeric so we can consider mean/median
# im considering median here

In [22]:
df['Loan_Amount_Term'].value_counts()

Loan_Amount_Term
360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
60.0       2
36.0       2
12.0       1
Name: count, dtype: int64

In [23]:
df['Loan_Amount_Term'].median()

np.float64(360.0)

In [24]:
df.where(df['LoanAmount'].isnull()).groupby('Loan_Amount_Term')[['LoanAmount','Credit_History','Loan_Status']].value_counts(dropna=False)

Loan_Amount_Term  LoanAmount  Credit_History  Loan_Status
180.0             NaN         1.0             N               2
240.0             NaN         1.0             Y               1
360.0             NaN         1.0             Y              10
                              0.0             N               4
                              1.0             N               4
                              NaN             N               1
Name: count, dtype: int64

In [25]:
# Imputing the null values in LoanAmount by analysing the Education level of the applicant

In [26]:
df.groupby('Education')[['LoanAmount']].mean()

,LoanAmount
Education,
Graduate,154.060215
Not Graduate,118.409449


In [27]:
df.groupby('Education')[['LoanAmount']].median()

,LoanAmount
Education,
Graduate,132.0
Not Graduate,115.0


In [28]:
# The MEDIAN of Graduate is 132 and Non-Graduate is 115
# Imputing the null value in LoanAmount column comparing against the education level's median
# Filling the Graduate applicant's NULL LoanAmount with the median LoanAmount of all the other Graduate applicants
# Filling the Non-Graduate applicant's NULL LoanAmount with the median LoanAmount of all the other Non-Graduate applicants

In [29]:
LoanAmountFill = []
for i,j in zip(df['LoanAmount'],df['Education']):
    if (pd.isna(i)) & (j == 'Graduate'):
        LoanAmountFill.append(df.where(df['Education']=='Graduate').groupby('Education')['LoanAmount'].median()[0])
    elif (pd.isna(i)) & (j == 'Not Graduate'):
        LoanAmountFill.append(df.where(df['Education']=='Not Graduate').groupby('Education')['LoanAmount'].median()[0])
    else:
        LoanAmountFill.append(i)
df['LoanAmountNULLFill'] = LoanAmountFill

In [30]:
df.where(df['Education']=='Graduate').groupby('Education')['LoanAmount'].median()[0]

np.float64(132.0)

In [31]:
len(LoanAmountFill)

614

In [32]:
df['LoanAmount'] = df['LoanAmountNULLFill']

In [33]:
df.drop('LoanAmountNULLFill',axis=1, inplace=True)

In [34]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Education             0
CoapplicantIncome     0
ApplicantIncome       0
Property_Area         0
LoanAmount            0
Loan_Status           0
Loan_Amount_Term     14
Dependents           15
Self_Employed        32
Credit_History       50
dtype: int64

In [35]:
## Imputed the LoanAmount column with predictive values

In [36]:
## imputing Loan_Amount_Term with repeated values

In [37]:
df['Loan_Amount_Term'].mode()[0]

np.float64(360.0)

In [38]:
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)

In [39]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Education             0
CoapplicantIncome     0
ApplicantIncome       0
Loan_Amount_Term      0
LoanAmount            0
Property_Area         0
Loan_Status           0
Dependents           15
Self_Employed        32
Credit_History       50
dtype: int64

In [40]:
## imputing Dependents with most repeated values of Dependents

In [41]:
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)

In [42]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Property_Area         0
Loan_Amount_Term      0
Loan_Status           0
Self_Employed        32
Credit_History       50
dtype: int64

In [43]:
## imputed NULL values of Dependents

In [44]:
## Imputing Null Values of Self_Employed with the most repeated values

In [45]:
df['Self_Employed'].mode()[0]

'No'

In [46]:
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)

In [47]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Property_Area         0
Loan_Status           0
Credit_History       50
dtype: int64

In [48]:
## imputed NULL values of Self_Employed

In [49]:
## Imputing NULL values of Credit_History with mode

In [50]:
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

In [51]:
df.isnull().sum().sort_values()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [52]:
## Imputed all the NULL values in the Dataset

In [53]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [54]:
df['LoanAmountNULLFill'] = LoanAmountFill

In [55]:
df['Education'].unique()

array(['Graduate', 'Not Graduate'], dtype=object)

In [56]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill
0,LP001002,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0


In [57]:
df['Education'].unique()

array(['Graduate', 'Not Graduate'], dtype=object)

In [58]:
df.where(df['Education']=='Graduate').groupby('Education')['LoanAmount'].median()[0]
# df.where(df['Education']=='Not Graduate').groupby('Education')['LoanAmount'].median()[0]

np.float64(132.0)

In [59]:
df.groupby('Property_Area')[['LoanAmount']].mean()

,LoanAmount
Property_Area,
Rural,151.391061
Semiurban,145.214592
Urban,141.222772


In [60]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill
0,LP001002,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0


In [61]:
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median(), inplace=True)

In [62]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
LoanAmountNULLFill    0
dtype: int64

In [63]:
## 4 -- Dependents check -- object/string -- going with mode

In [64]:
df['Dependents'].value_counts(dropna=False)

Dependents
0     360
1     102
2     101
3+     51
Name: count, dtype: int64

In [65]:
type(df['Dependents'])

pandas.core.series.Series

In [66]:
df['Dependents'].mode()[0]

'0'

In [67]:
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)

In [68]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
LoanAmountNULLFill    0
dtype: int64

In [69]:
## 5 -- Loan Amount check -- numeric -- going with mode/median

In [70]:
df['LoanAmount'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
LoanAmount,614.0,145.701954,84.131689,9.0,100.25,128.0,164.75,700.0


In [71]:
## it is not normal distribution so im not considering mean here, instead considering MEDIAN

In [72]:
df['LoanAmount'].median()

np.float64(128.0)

In [73]:
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)

In [74]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
LoanAmountNULLFill    0
dtype: int64

In [75]:
## 6 -- Self_Employed check -- object/string -- going with MODE

In [76]:
df['Self_Employed'].value_counts(dropna=False)

Self_Employed
No     532
Yes     82
Name: count, dtype: int64

In [77]:
df['Self_Employed'].mode()[0]

'No'

In [78]:
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)

In [79]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
LoanAmountNULLFill    0
dtype: int64

In [80]:
## 7 -- Credit_History check -- numeric -- going with mean/median

In [81]:
df['Credit_History'].value_counts(dropna=False)

Credit_History
1.0    525
0.0     89
Name: count, dtype: int64

In [82]:
df['Credit_History'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
Credit_History,614.0,0.855049,0.352339,0.0,1.0,1.0,1.0,1.0


In [83]:
df.groupby('Credit_History')['Loan_Status'].value_counts()
# even for a person with credit history - loan is approved and rejected so i can proceed with median

Credit_History  Loan_Status
0.0             N               82
                Y                7
1.0             Y              415
                N              110
Name: count, dtype: int64

In [84]:
df['Credit_History'].median()

np.float64(1.0)

In [85]:
df['Credit_History'].fillna(df['Credit_History'].median(), inplace=True)

In [86]:
df.isnull().sum().sort_values()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
LoanAmountNULLFill    0
dtype: int64

## ------ All null values are filled/imputed ------

## Checking for spaces in column data

In [87]:
df['Gender'].unique()

array(['Male', 'Female'], dtype=object)

## Univariate Analysis

In [88]:
df.groupby('Gender')['Loan_Status'].value_counts(normalize=True)*100

Gender  Loan_Status
Female  Y              66.964286
        N              33.035714
Male    Y              69.123506
        N              30.876494
Name: proportion, dtype: float64

In [89]:
## Equal no.of Approvals and Rejections for both gender

In [90]:
df.groupby('Married')['Loan_Status'].value_counts(normalize=True)*100

Married  Loan_Status
No       Y              62.910798
         N              37.089202
Yes      Y              71.820449
         N              28.179551
Name: proportion, dtype: float64

In [91]:
## Approval rates for Married people is slightly higher - 71% for married and 62% for non-married

In [92]:
df.groupby('Dependents')['Loan_Status'].value_counts(normalize=True)*100

Dependents  Loan_Status
0           Y              68.611111
            N              31.388889
1           Y              64.705882
            N              35.294118
2           Y              75.247525
            N              24.752475
3+          Y              64.705882
            N              35.294118
Name: proportion, dtype: float64

In [93]:
# Approval rate for having 2 dependents is 75% while the others is between 645 to 69%

In [94]:
df.groupby('Education')['Loan_Status'].value_counts(normalize=True)*100

Education     Loan_Status
Graduate      Y              70.833333
              N              29.166667
Not Graduate  Y              61.194030
              N              38.805970
Name: proportion, dtype: float64

In [95]:
# Approval rate for graduates is 70% and non-graduates is 61%

In [96]:
df.groupby('Self_Employed')['Loan_Status'].value_counts(normalize=True)*100

Self_Employed  Loan_Status
No             Y              68.796992
               N              31.203008
Yes            Y              68.292683
               N              31.707317
Name: proportion, dtype: float64

In [97]:
# not much of a difference

In [98]:
df.groupby('ApplicantIncome')['Loan_Status'].value_counts(normalize=True)*100

ApplicantIncome  Loan_Status
150              N              100.0
210              Y              100.0
416              N              100.0
645              Y              100.0
674              Y              100.0
                                ...  
39147            Y              100.0
39999            Y              100.0
51763            Y              100.0
63337            Y              100.0
81000            N              100.0
Name: proportion, Length: 536, dtype: float64

In [99]:
df['ApplicantIncome'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
ApplicantIncome,614.0,5403.459283,6109.041673,150.0,2877.5,3812.5,5795.0,81000.0


In [100]:
AI_Group = []
for i in df['ApplicantIncome']:
    if i <= np.percentile(df['ApplicantIncome'],q=25):
        AI_Group.append('Group A')
    elif np.percentile(df['ApplicantIncome'],q=25) <i <=np.percentile(df['ApplicantIncome'],q=50):
        AI_Group.append('Group B')
    elif np.percentile(df['ApplicantIncome'],q=50) <i <=np.percentile(df['ApplicantIncome'],q=75):
        AI_Group.append('Group C')
    elif i > np.percentile(df['ApplicantIncome'],q=75):
        AI_Group.append('Group D')
df['AI_Group'] = AI_Group

In [101]:
df.groupby('AI_Group')['Loan_Status'].value_counts(normalize=True)*100

AI_Group  Loan_Status
Group A   Y              68.831169
          N              31.168831
Group B   Y              68.627451
          N              31.372549
Group C   Y              69.281046
          N              30.718954
Group D   Y              68.181818
          N              31.818182
Name: proportion, dtype: float64

In [102]:
## not much of a difference

In [103]:
df.groupby('CoapplicantIncome')['Loan_Status'].value_counts(normalize=True)*100

CoapplicantIncome  Loan_Status
0.000000           Y               64.835165
                   N               35.164835
16.120001          Y              100.000000
189.000000         Y              100.000000
240.000000         Y              100.000000
                                     ...    
11300.000000       N              100.000000
20000.000000       N               50.000000
                   Y               50.000000
33837.000000       N              100.000000
41667.000000       N              100.000000
Name: proportion, Length: 303, dtype: float64

In [104]:
df['CoapplicantIncome'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
CoapplicantIncome,614.0,1621.245798,2926.248369,0.0,0.0,1188.5,2297.25,41667.0


In [105]:
CoI_Group = []
for i in df['CoapplicantIncome']:
    if i <= np.percentile(df['CoapplicantIncome'],q=25):
        CoI_Group.append('Group A')
    elif np.percentile(df['CoapplicantIncome'],q=25) <i <=np.percentile(df['CoapplicantIncome'],q=50):
        CoI_Group.append('Group B')
    elif np.percentile(df['CoapplicantIncome'],q=50) <i <=np.percentile(df['CoapplicantIncome'],q=75):
        CoI_Group.append('Group C')
    elif i > np.percentile(df['CoapplicantIncome'],q=75):
        CoI_Group.append('Group D')
df['CoI_Group'] = CoI_Group

In [106]:
df.groupby('CoI_Group')['Loan_Status'].value_counts(normalize=True)*100

CoI_Group  Loan_Status
Group A    Y              64.835165
           N              35.164835
Group B    Y              91.176471
           N               8.823529
Group C    Y              70.588235
           N              29.411765
Group D    Y              68.831169
           N              31.168831
Name: proportion, dtype: float64

In [107]:
## 91% of the Co-applicants who earn between 0 to 1188.5 have been Approved, others as usual 65/35 dist

In [108]:
df['CombinedIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']

In [109]:
df['CombinedIncome'] = df['CombinedIncome'].astype(int)

In [110]:
df.groupby('CombinedIncome')['Loan_Status']

In [111]:
# Creating a range for the combined income and then grouping them

In [112]:
df['CombinedIncome'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
CombinedIncome,614.0,7024.703583,6458.664751,1442.0,4166.0,5416.5,7521.75,81000.0


In [113]:
np.percentile(df['CombinedIncome'],q=50)

np.float64(5416.5)

In [114]:
CI_Group = []
for i in df['CombinedIncome']:
    if i <= np.percentile(df['CombinedIncome'],q=25):
        CI_Group.append('Group A')
    elif np.percentile(df['CombinedIncome'],q=25) <i <=np.percentile(df['CombinedIncome'],q=50):
        CI_Group.append('Group B')
    elif np.percentile(df['CombinedIncome'],q=50) <i <=np.percentile(df['CombinedIncome'],q=75):
        CI_Group.append('Group C')
    elif i > np.percentile(df['CombinedIncome'],q=75):
        CI_Group.append('Group D')
df['CI_Group'] = CI_Group

In [115]:
df.groupby('CI_Group')['Loan_Status'].value_counts(normalize=True)*100

CI_Group  Loan_Status
Group A   Y              67.741935
          N              32.258065
Group B   Y              68.421053
          N              31.578947
Group C   Y              71.241830
          N              28.758170
Group D   Y              67.532468
          N              32.467532
Name: proportion, dtype: float64

In [116]:
## not much of a huge diff

In [121]:
df['Loan_Amount_Term_Year'] = df['Loan_Amount_Term']/12

In [122]:
df.groupby('Loan_Amount_Term_Year')['Loan_Status'].value_counts(normalize=True)*100

Loan_Amount_Term_Year  Loan_Status
1.0                    Y              100.000000
3.0                    N              100.000000
5.0                    Y              100.000000
7.0                    Y               75.000000
                       N               25.000000
10.0                   Y              100.000000
15.0                   Y               65.909091
                       N               34.090909
20.0                   Y               75.000000
                       N               25.000000
25.0                   Y               61.538462
                       N               38.461538
30.0                   Y               69.771863
                       N               30.228137
40.0                   N               60.000000
                       Y               40.000000
Name: proportion, dtype: float64

In [123]:
df.rename(columns = {'Loan_Amount_Term':'Loan_Amount_Term_Month'}, inplace=True)

In [124]:
df.groupby('Loan_Amount_Term_Month')['Loan_Status'].value_counts(normalize=True)*100

Loan_Amount_Term_Month  Loan_Status
12.0                    Y              100.000000
36.0                    N              100.000000
60.0                    Y              100.000000
84.0                    Y               75.000000
                        N               25.000000
120.0                   Y              100.000000
180.0                   Y               65.909091
                        N               34.090909
240.0                   Y               75.000000
                        N               25.000000
300.0                   Y               61.538462
                        N               38.461538
360.0                   Y               69.771863
                        N               30.228137
480.0                   N               60.000000
                        Y               40.000000
Name: proportion, dtype: float64

In [125]:
L1 = pd.DataFrame(df.groupby('Loan_Amount_Term_Year')[['Education','Credit_History','Loan_Status']].value_counts(normalize=False))
L1['%'] = round(df.groupby('Loan_Amount_Term_Year')[['Education','Credit_History','Loan_Status']].value_counts(normalize=True)*100,2)
L1

count       %
Loan_Amount_Term_Year Education    Credit_History Loan_Status               
1.0                   Graduate     1.0            Y                1  100.00
3.0                   Graduate     1.0            N                1   50.00
                      Not Graduate 1.0            N                1   50.00
5.0                   Graduate     1.0            Y                1   50.00
                      Not Graduate 1.0            Y                1   50.00
7.0                   Graduate     1.0            Y                3   75.00
                                                  N                1   25.00
10.0                  Graduate     1.0            Y                2   66.67
                      Not Graduate 1.0            Y                1   33.33
15.0                  Graduate     1.0            Y               20   45.45
                      Not Graduate 1.0            Y                7   15.91
                                   0.0            N                5   11.36
                                   1.0            N                4    9.09
                      Graduate     0.0            N                3    6.82
                                   1.0            N                3    6.82
                                   0.0            Y                2    4.55
20.0                  Graduate     1.0            Y                2   50.00
                                                  N                1   25.00
                      Not Graduate 1.0            Y                1   25.00
25.0                  Graduate     1.0            Y                6   46.15
                                   0.0            N                3   23.08
                      Not Graduate 1.0            Y                2   15.38
                      Graduate     1.0            N                1    7.69
                      Not Graduate 1.0            N                1    7.69
30.0                  Graduate     1.0            Y              294   55.89
                                                  N               72   13.69
                      Not Graduate 1.0            Y               68   12.93
                      Graduate     0.0            N               49    9.32
                      Not Graduate 1.0            N               20    3.80
                                   0.0            N               18    3.42
                      Graduate     0.0            Y                4    0.76
                      Not Graduate 0.0            Y                1    0.19
40.0                  Graduate     1.0            Y                5   33.33
                                                  N                4   26.67
                                   0.0            N                2   13.33
                      Not Graduate 0.0            N                2   13.33
                                   1.0            N                1    6.67
                                                  Y                1    6.67

In [126]:
df.where(df['Loan_Amount_Term_Year']==3).groupby('Loan_Amount_Term_Year')[['Married','Education','Dependents','Credit_History','Loan_Status']].value_counts(normalize=True)*100

Loan_Amount_Term_Year  Married  Education     Dependents  Credit_History  Loan_Status
3.0                    No       Graduate      1           1.0             N              50.0
                                Not Graduate  0           1.0             N              50.0
Name: proportion, dtype: float64

In [127]:
## person is not married so loan rejected for 3 year term loan

In [128]:
df.where(df['Loan_Amount_Term_Year']==7).groupby('Loan_Amount_Term_Year')[['Married','Education','Dependents','Self_Employed','Property_Area','Credit_History','Loan_Status']].value_counts(normalize=True)*100

Loan_Amount_Term_Year  Married  Education  Dependents  Self_Employed  Property_Area  Credit_History  Loan_Status
7.0                    Yes      Graduate   1           No             Rural          1.0             N              25.0
                                                                      Semiurban      1.0             Y              25.0
                                           2           No             Urban          1.0             Y              25.0
                                                       Yes            Rural          1.0             Y              25.0
Name: proportion, dtype: float64

In [129]:
## Applicant from rural is rejected here

In [130]:
df.where(df['Loan_Amount_Term_Year']==10).groupby('Loan_Amount_Term_Year')[['Married','Education','Dependents','Self_Employed','Property_Area','Credit_History','Loan_Status']].value_counts(normalize=True)*100

Loan_Amount_Term_Year  Married  Education     Dependents  Self_Employed  Property_Area  Credit_History  Loan_Status
10.0                   No       Not Graduate  0           No             Semiurban      1.0             Y              33.333333
                       Yes      Graduate      0           Yes            Semiurban      1.0             Y              33.333333
                                              2           No             Urban          1.0             Y              33.333333
Name: proportion, dtype: float64

In [131]:
df.groupby('Property_Area')['LoanAmount'].median(), df['Property_Area'].unique()

(Property_Area
 Rural        132.0
 Semiurban    128.0
 Urban        120.0
 Name: LoanAmount, dtype: float64,
 array(['Urban', 'Rural', 'Semiurban'], dtype=object))

In [132]:
df.where((df['Loan_Amount_Term_Year']==15) & (df['Property_Area']=='Rural')).groupby('Loan_Amount_Term_Year')[['Married','Education','Dependents','Self_Employed','LoanAmount','Property_Area','Credit_History','Loan_Status']].value_counts(normalize=True)*100

Loan_Amount_Term_Year  Married  Education     Dependents  Self_Employed  LoanAmount  Property_Area  Credit_History  Loan_Status
15.0                   Yes      Graduate      0           No             42.0        Rural          1.0             Y              9.090909
                                                                         90.0        Rural          0.0             Y              9.090909
                                                                         132.0       Rural          0.0             N              9.090909
                                                                         142.0       Rural          1.0             Y              9.090909
                                                                         188.0       Rural          1.0             Y              9.090909
                                              1           No             185.0       Rural          1.0             Y              9.090909
                                

In [135]:
pd.set_option('display.max_rows',None)

In [136]:
LoanAmountBucket = []
for i in df['LoanAmount']:
    if (i < np.percentile(df['LoanAmount'],q=25)):
        LoanAmountBucket.append('< 100.25')
    elif (np.percentile(df['LoanAmount'],q=25) <= i < np.percentile(df['LoanAmount'],q=50)):
        LoanAmountBucket.append('100.25 to 128.0')
    elif (np.percentile(df['LoanAmount'],q=50) <= i < np.percentile(df['LoanAmount'],q=75)):
        LoanAmountBucket.append('128.0 to 164.75')
    elif (np.percentile(df['LoanAmount'],q=75) <= i):
        LoanAmountBucket.append('164.75 to 700')

In [138]:
df['LoanAmountBucket'] = LoanAmountBucket

In [139]:
Area = pd.DataFrame(df.groupby(['Property_Area','Education','Credit_History','CI_Group','LoanAmountBucket'])['Loan_Status'].value_counts(normalize=False))
Area['%'] = round(df.groupby(['Property_Area','Education','Credit_History','CI_Group','LoanAmountBucket'])['Loan_Status'].value_counts(normalize=True)*100,2)
Area

count  \
Property_Area Education    Credit_History CI_Group LoanAmountBucket Loan_Status          
Rural         Graduate     0.0            Group A  100.25 to 128.0  N                1   
                                                   < 100.25         N                2   
                                          Group B  100.25 to 128.0  N                1   
                                                   128.0 to 164.75  N                3   
                                                                    Y                1   
                                                   < 100.25         Y                1   
                                          Group C  100.25 to 128.0  N                5   
                                                   128.0 to 164.75  N                2   
                                          Group D  128.0 to 164.75  N                1   
                                                   164.75 to 700    N                3   
                           1.0            Group A  100.25 to 128.0  Y                4   
                                                   128.0 to 164.75  N                2   
                                                   164.75 to 700    N                1   
                                                                    Y                1   
                                                   < 100.25         Y                6   
                                                                    N                2   
                                          Group B  100.25 to 128.0  Y                7   
                                                                    N                3   
                                                   128.0 to 164.75  Y                6   
                                                                    N                1   
                                                   < 100.25         N                2   
                                                                    Y                2   
                                          Group C  100.25 to 128.0  Y                5   
                                                   128.0 to 164.75  Y               17   
                                                                    N                2   
                                                   164.75 to 700    Y                6   
                                                                    N                3   
                                                   < 100.25         Y                2   
                                                                    N                1   
                                          Group D  100.25 to 128.0  Y                2   
                                                   128.0 to 164.75  Y                4   
                                                                    N                3   
                                                   164.75 to 700    Y               20   
                                                                    N                9   
              Not Graduate 0.0            Group A  100.25 to 128.0  N                2   
                                          Group B  100.25 to 128.0  N                1   
                                          Group C  100.25 to 128.0  N                1   
                                                   128.0 to 164.75  N                1   
                                                   164.75 to 700    N                2   
                                          Group D  128.0 to 164.75  N                1   
                           1.0            Group A  100.25 to 128.0  Y                4   
                                                                    N                1   
                                                   < 100.25         N                4   
                               

In [140]:
CRL = pd.DataFrame(df.groupby(['Credit_History','Education','LoanAmountBucket'])['Loan_Status'].value_counts(normalize=False))
CRL['%'] = round(df.groupby(['Credit_History','Education','LoanAmountBucket'])['Loan_Status'].value_counts(normalize=True)*100,2)
CRL

count       %
Credit_History Education    LoanAmountBucket Loan_Status               
0.0            Graduate     100.25 to 128.0  N               18  100.00
                            128.0 to 164.75  N               15   83.33
                                             Y                3   16.67
                            164.75 to 700    N               13   86.67
                                             Y                2   13.33
                            < 100.25         N               11   91.67
                                             Y                1    8.33
               Not Graduate 100.25 to 128.0  N               11  100.00
                            128.0 to 164.75  N                3   75.00
                                             Y                1   25.00
                            164.75 to 700    N                5  100.00
                            < 100.25         N                6  100.00
1.0            Graduate     100.25 to 128.0  Y               70   86.42
                                             N               11   13.58
                            128.0 to 164.75  Y               91   82.73
                                             N               19   17.27
                            164.75 to 700    Y               92   73.60
                                             N               33   26.40
                            < 100.25         Y               81   80.20
                                             N               20   19.80
               Not Graduate 100.25 to 128.0  Y               33   86.84
                                             N                5   13.16
                            128.0 to 164.75  Y               18   69.23
                                             N                8   30.77
                            164.75 to 700    Y                5   55.56
                                             N                4   44.44
                            < 100.25         Y               25   71.43
                                             N               10   28.57

In [141]:
## Non-Graduate and No Credit History are 90-100% rejected from all areas

In [142]:
## Loan Amount Group creation

In [143]:
df['LoanAmount'].describe().reset_index().transpose()

,0,1,2,3,4,5,6,7
index,count,mean,std,min,25%,50%,75%,max
LoanAmount,614.0,145.701954,84.131689,9.0,100.25,128.0,164.75,700.0


In [144]:
np.percentile(df['LoanAmount'],q=25)/2, np.percentile(df['LoanAmount'],q=50)

(np.float64(50.125), np.float64(128.0))

In [145]:
np.percentile(df['LoanAmount'],q=25)/2

np.float64(50.125)

In [146]:
print('<',np.percentile(df['LoanAmount'],q=50))

< 128.0


In [147]:
df['Loan_Status'].value_counts(normalize=True)*100

Loan_Status
Y    68.729642
N    31.270358
Name: proportion, dtype: float64

In [148]:
df.groupby('Credit_History')['Loan_Status'].value_counts(normalize=True)*100

Credit_History  Loan_Status
0.0             N              92.134831
                Y               7.865169
1.0             Y              79.047619
                N              20.952381
Name: proportion, dtype: float64

In [ ]:
# more rejections for people with NO credit history (92%)

In [149]:
df.groupby('Loan_Status')['LoanAmount'].mean()

Loan_Status
N    149.677083
Y    143.893365
Name: LoanAmount, dtype: float64

In [150]:
df.groupby('Property_Area')['Loan_Status'].value_counts(normalize=True)*100

Property_Area  Loan_Status
Rural          Y              61.452514
               N              38.547486
Semiurban      Y              76.824034
               N              23.175966
Urban          Y              65.841584
               N              34.158416
Name: proportion, dtype: float64

In [ ]:
# semi-urban approval rate is slightly more compared to other areas

## ML Model

In [151]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#### Creating a DataFrame for ML by copying the data from df

In [152]:
ml = df.copy()

In [153]:
ml.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,AI_Group,CoI_Group,CombinedIncome,CI_Group,Loan_Amount_Term_Year,LoanAmountBucket
0,LP001002,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0,Group D,Group A,5849,Group C,30.0,128.0 to 164.75
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0,Group C,Group C,6091,Group C,30.0,128.0 to 164.75
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0,Group B,Group A,3000,Group A,30.0,< 100.25
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0,Group A,Group D,4941,Group B,30.0,100.25 to 128.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0,Group D,Group A,6000,Group C,30.0,128.0 to 164.75


In [155]:
Loan_ID = ml['Loan_ID']

In [156]:
Loan_ID = Loan_ID.to_list()

In [157]:
type(Loan_ID)

list

In [158]:
ml.drop('Loan_ID',axis=1,inplace=True)

In [159]:
ml.drop(['AI_Group','CoI_Group','CI_Group','Loan_Amount_Term_Year','LoanAmountBucket'],axis=1, inplace=True)

In [160]:
ml.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,CombinedIncome
0,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0,5849
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0,6091
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0,3000
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0,4941
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0,6000


## PreProcessing

### LabelEncoding, OrdinalEncoding

In [161]:
## Label encoding --> Gender, Married, Self_Employed
## Manual map --> Loan_Status
## Ordinal encoding --> Education, Property_Area

In [162]:
le_g = LabelEncoder()
le_m = LabelEncoder()
le_se = LabelEncoder()
oe_e = OrdinalEncoder(categories=[['Not Graduate','Graduate']])
oe_pa = OrdinalEncoder(categories=[['Rural','Semiurban','Urban']])

In [163]:
ml['Gender'] = le_g.fit_transform(ml['Gender'])
ml['Married'] = le_m.fit_transform(ml['Married'])
ml['Self_Employed'] = le_se.fit_transform(ml['Self_Employed'])

In [164]:
ml.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,CombinedIncome
0,1,0,0,Graduate,0,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0,5849
1,1,1,1,Graduate,0,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0,6091
2,1,1,0,Graduate,1,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0,3000
3,1,1,0,Not Graduate,0,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0,4941
4,1,0,0,Graduate,0,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0,6000


In [165]:
ml['Property_Area'].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [166]:
ml['Education'] = oe_e.fit_transform(ml[['Education']])
ml['Property_Area'] = oe_pa.fit_transform(ml[['Property_Area']])

In [167]:
ml.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,CombinedIncome
0,1,0,0,1.0,0,5849,0.0,132.0,360.0,1.0,2.0,Y,132.0,5849
1,1,1,1,1.0,0,4583,1508.0,128.0,360.0,1.0,0.0,N,128.0,6091
2,1,1,0,1.0,1,3000,0.0,66.0,360.0,1.0,2.0,Y,66.0,3000
3,1,1,0,0.0,0,2583,2358.0,120.0,360.0,1.0,2.0,Y,120.0,4941
4,1,0,0,1.0,0,6000,0.0,141.0,360.0,1.0,2.0,Y,141.0,6000


In [168]:
ml['Loan_Status'] = ml['Loan_Status'].map({'Y':1, 'N':0})

In [169]:
ml.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,CombinedIncome
0,1,0,0,1.0,0,5849,0.0,132.0,360.0,1.0,2.0,1,132.0,5849
1,1,1,1,1.0,0,4583,1508.0,128.0,360.0,1.0,0.0,0,128.0,6091
2,1,1,0,1.0,1,3000,0.0,66.0,360.0,1.0,2.0,1,66.0,3000
3,1,1,0,0.0,0,2583,2358.0,120.0,360.0,1.0,2.0,1,120.0,4941
4,1,0,0,1.0,0,6000,0.0,141.0,360.0,1.0,2.0,1,141.0,6000


In [170]:
ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Gender                  614 non-null    int64  
 1   Married                 614 non-null    int64  
 2   Dependents              614 non-null    object 
 3   Education               614 non-null    float64
 4   Self_Employed           614 non-null    int64  
 5   ApplicantIncome         614 non-null    int64  
 6   CoapplicantIncome       614 non-null    float64
 7   LoanAmount              614 non-null    float64
 8   Loan_Amount_Term_Month  614 non-null    float64
 9   Credit_History          614 non-null    float64
 10  Property_Area           614 non-null    float64
 11  Loan_Status             614 non-null    int64  
 12  LoanAmountNULLFill      614 non-null    float64
 13  CombinedIncome          614 non-null    int64  
dtypes: float64(7), int64(6), object(1)
memory 

In [171]:
ml['Dependents'].unique()

array(['0', '1', '2', '3+'], dtype=object)

In [172]:
## Oridnal encoding for Dependents

In [173]:
oe_d = OrdinalEncoder(categories=[['0','1','2','3+']])

In [174]:
ml['Dependents'] = oe_d.fit_transform(ml[['Dependents']])

In [175]:
ml.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,CombinedIncome
0,1,0,0.0,1.0,0,5849,0.0,132.0,360.0,1.0,2.0,1,132.0,5849
1,1,1,1.0,1.0,0,4583,1508.0,128.0,360.0,1.0,0.0,0,128.0,6091
2,1,1,0.0,1.0,1,3000,0.0,66.0,360.0,1.0,2.0,1,66.0,3000
3,1,1,0.0,0.0,0,2583,2358.0,120.0,360.0,1.0,2.0,1,120.0,4941
4,1,0,0.0,1.0,0,6000,0.0,141.0,360.0,1.0,2.0,1,141.0,6000


In [176]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,Loan_Status,LoanAmountNULLFill,AI_Group,CoI_Group,CombinedIncome,CI_Group,Loan_Amount_Term_Year,LoanAmountBucket
0,LP001002,Male,No,0,Graduate,No,5849,0.0,132.0,360.0,1.0,Urban,Y,132.0,Group D,Group A,5849,Group C,30.0,128.0 to 164.75
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,128.0,Group C,Group C,6091,Group C,30.0,128.0 to 164.75
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,66.0,Group B,Group A,3000,Group A,30.0,< 100.25
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,120.0,Group A,Group D,4941,Group B,30.0,100.25 to 128.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,141.0,Group D,Group A,6000,Group C,30.0,128.0 to 164.75


## Train Test Split

In [177]:
x = ml.drop('Loan_Status', axis=1)
y = ml[['Loan_Status']]

In [178]:
x.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term_Month,Credit_History,Property_Area,LoanAmountNULLFill,CombinedIncome
0,1,0,0.0,1.0,0,5849,0.0,132.0,360.0,1.0,2.0,132.0,5849
1,1,1,1.0,1.0,0,4583,1508.0,128.0,360.0,1.0,0.0,128.0,6091
2,1,1,0.0,1.0,1,3000,0.0,66.0,360.0,1.0,2.0,66.0,3000
3,1,1,0.0,0.0,0,2583,2358.0,120.0,360.0,1.0,2.0,120.0,4941
4,1,0,0.0,1.0,0,6000,0.0,141.0,360.0,1.0,2.0,141.0,6000


In [179]:
y.head()

,Loan_Status
0,1
1,0
2,1
3,1
4,1


In [180]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.05, random_state=42)

## Logistics Regression

In [181]:
lr = LogisticRegression()

In [182]:
lr.fit(xtrain, ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [183]:
lr_preds = lr.predict(xtest)

In [184]:
lr_acc = accuracy_score(ytest, lr_preds)
lr_acc

0.9032258064516129

In [185]:
lr_conf = confusion_matrix(ytest, lr_preds)
lr_conf

array([[ 1,  3],
       [ 0, 27]])

In [186]:
lr_cr = classification_report(ytest, lr_preds)
print(lr_cr)

              precision    recall  f1-score   support

           0       1.00      0.25      0.40         4
           1       0.90      1.00      0.95        27

    accuracy                           0.90        31
   macro avg       0.95      0.62      0.67        31
weighted avg       0.91      0.90      0.88        31



## K Nearest Neigbour

In [187]:
knn = KNeighborsClassifier()

In [188]:
knn.fit(xtrain, ytrain)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [189]:
knn_preds = knn.predict(xtest)

In [190]:
knn_acc = accuracy_score(ytest, knn_preds)
knn_acc

0.8064516129032258

In [191]:
knn_conf = confusion_matrix(ytest, knn_preds)
knn_conf

array([[ 1,  3],
       [ 3, 24]])

In [192]:
knn_class = classification_report(ytest, knn_preds)
print(knn_class)

              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.89      0.89      0.89        27

    accuracy                           0.81        31
   macro avg       0.57      0.57      0.57        31
weighted avg       0.81      0.81      0.81        31



## Decision Tree

In [193]:
dt = DecisionTreeClassifier()

In [194]:
dt.fit(xtrain, ytrain)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [195]:
dt_preds = dt.predict(xtest)

In [196]:
dt_acc = accuracy_score(ytest, dt_preds)
dt_acc

0.7741935483870968

In [197]:
dt_conf = confusion_matrix(ytest, dt_preds)
dt_conf

array([[ 3,  1],
       [ 6, 21]])

In [198]:
dt_class = classification_report(ytest, dt_preds)
print(dt_class)

              precision    recall  f1-score   support

           0       0.33      0.75      0.46         4
           1       0.95      0.78      0.86        27

    accuracy                           0.77        31
   macro avg       0.64      0.76      0.66        31
weighted avg       0.87      0.77      0.81        31



## Random Forest

In [199]:
rf = RandomForestClassifier()

In [200]:
rf.fit(xtrain, ytrain)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [201]:
rf_preds = rf.predict(xtest)

In [202]:
rf_acc = accuracy_score(ytest, rf_preds)
rf_acc

0.9032258064516129

In [203]:
rf_conf = confusion_matrix(ytest, rf_preds)
rf_conf

array([[ 1,  3],
       [ 0, 27]])

In [204]:
rf_class = classification_report(ytest, rf_preds)
print(rf_class)

              precision    recall  f1-score   support

           0       1.00      0.25      0.40         4
           1       0.90      1.00      0.95        27

    accuracy                           0.90        31
   macro avg       0.95      0.62      0.67        31
weighted avg       0.91      0.90      0.88        31



### Decision Tree despite having low precsion has done well on actually rejecting those that should have been rejected for a loan as that makes it more dangerous (our Type 2 error). Hence, I have considered DT to be my best performing model!

In [205]:
dt_conf

array([[ 3,  1],
       [ 6, 21]])